In [1]:
import numpy as np
%cd  '/home/ubuntu/image-drift-monitoring/data/camelyon17_v1.0'
%pwd

camelyon_train_comp = np.load('camelyon_train_ds.npz')

/home/ubuntu/image-drift-monitoring/data/camelyon17_v1.0


In [2]:
camelyon_train = camelyon_train_comp['arr_0']

np.random.shuffle(camelyon_train)

In [8]:
# np.random.shuffle(camelyon_train)
# camelyon_train_5  = camelyon_train[:int(len(camelyon_train)/20)]
np.random.shuffle(camelyon_train)
camelyon_train_10 = camelyon_train[:int(len(camelyon_train)/10)]
# np.random.shuffle(camelyon_train)
# camelyon_train_15 = camelyon_train[:int(len(camelyon_train)/(20/3))]
# np.random.shuffle(camelyon_train)
# camelyon_train_20 = camelyon_train[:int(len(camelyon_train)/5)]
# np.random.shuffle(camelyon_train)
# camelyon_train_25 = camelyon_train[:int(len(camelyon_train)/4)]
# np.random.shuffle(camelyon_train)
# camelyon_train_30 = camelyon_train[:int(len(camelyon_train)/(10/3))]
# np.random.shuffle(camelyon_train)
# camelyon_train_35 = camelyon_train[:int(len(camelyon_train)/(20/7))]
# np.random.shuffle(camelyon_train)
# camelyon_train_40 = camelyon_train[:int(len(camelyon_train)/(5/2))]
# np.random.shuffle(camelyon_train)
# camelyon_train_45 = camelyon_train[:int(len(camelyon_train)/(20/9))]
# np.random.shuffle(camelyon_train)
# camelyon_train_50 = camelyon_train[:int(len(camelyon_train)/2)]
# np.random.shuffle(camelyon_train)
# camelyon_train_55 = camelyon_train[:int(len(camelyon_train)/(20/11))]
# np.random.shuffle(camelyon_train)
# camelyon_train_60 = camelyon_train[:int(len(camelyon_train)*0.6)]
# np.random.shuffle(camelyon_train)
# camelyon_train_65 = camelyon_train[:int(len(camelyon_train)*0.65)]
# np.random.shuffle(camelyon_train)
# camelyon_train_70 = camelyon_train[:int(len(camelyon_train)*0.7)]
# np.random.shuffle(camelyon_train)
# camelyon_train_75 = camelyon_train[:int(len(camelyon_train)*0.75)]
# np.random.shuffle(camelyon_train)
# camelyon_train_80 = camelyon_train[:int(len(camelyon_train)*0.8)]
# np.random.shuffle(camelyon_train)
# camelyon_train_85 = camelyon_train[:int(len(camelyon_train)*0.85)]
# np.random.shuffle(camelyon_train)
# camelyon_train_90 = camelyon_train[:int(len(camelyon_train)*0.9)]
# np.random.shuffle(camelyon_train)
# camelyon_train_95 = camelyon_train[:int(len(camelyon_train)*0.95)]
# np.random.shuffle(camelyon_train)

In [ ]:
camelyon_train.shape

In [9]:
camelyon_train_10.shape

(30243, 3, 96, 96)

In [10]:
import json 
with open('/home/ubuntu/image-drift-monitoring/config/common/drift_detection_config.json') as config_file:
        drift_detection_config = json.load(config_file)

In [11]:
%cd  '/home/ubuntu/image-drift-monitoring'
%pwd
from src.modules.alibi_detect.untrained_encoder import UntrainedAutoencoder

myUAE = UntrainedAutoencoder(drift_detection_config)

/home/ubuntu/image-drift-monitoring


In [12]:
myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_10.shape[1:])
myUAE.init_detector(detector_type='CVM',reference_data=camelyon_train_10,detector_name='Camelyon_UAE_10_CVM',save_dec=True)

2023-02-10 15:34:06,336 [INFO]  CVM Detector initialized
2023-02-10 15:34:06,336 [INFO]  CVM Detector initialized
Directory /home/ubuntu/image-drift-monitoring/config/detectors/Camelyon/CVM/Camelyon_UAE_10_CVM does not exist and is now created.


# H0 Hypothesis testing

In [ ]:
myUAE.make_prediction(target_data=camelyon_train_12_5_25,detector_type='KS')

# drift testing

In [ ]:
camelyon_test_comp = np.load('/home/ubuntu/image-drift-monitoring/data/camelyon17_v1.0/camelyon_test_ds.npz')
camelyon_test = camelyon_test_comp['arr_0']

np.random.shuffle(camelyon_test)

In [ ]:
myUAE.make_prediction(target_data=camelyon_test,detector_type='KS')